In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA está disponible en este sistema.")
    print("Número de GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA NO está disponible.")

CUDA está disponible en este sistema.
Número de GPUs: 1
GPU 0: NVIDIA GeForce RTX 4070 Ti


In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

data_url = 'data/data_processed.parquet'
data = TabularDataset(data_url)
data = data.loc[:, [col for col in data.columns if col not in ['product_id', 'title']]]
data.head()

,condition,state,city,local_pickup,free_shipping,shipping_mode,listing_type,buying_mode,attribute_group_id,attribute_group,attribute_id,status,accepts_mercadopago,currency,automatic_relist,stock_quantity,available_quantity,total_amount,date_difference_hr,time_difference_hr
0,new,buenos aires,ciudad madero,True,False,me2,gold,buy_it_now,find,ficha técnica,season,active,True,ars,False,82944,46656,3.640412e+08,420.406111,1440.0
1,new,capital federal,belgrano,True,False,me2,silver,buy_it_now,dflt,ficha técnica,season,active,True,ars,True,27456,10560,1.298880e+08,1007.802222,1440.0
2,new,capital federal,belgrano r,True,False,me2,silver,buy_it_now,dflt,ficha técnica,season,active,True,ars,False,32832,27360,1.045683e+08,1182.258333,1440.0
3,new,buenos aires,san justo,True,False,me2,gold,buy_it_now,dflt,otros,season,active,True,ars,False,90720,89376,6.378893e+07,96.116389,1440.0
4,new,capital federal,chacarita,False,False,me2,gold_special,buy_it_now,dflt,otros,gender,active,True,ars,True,38880,31104,5.566450e+07,340.913333,1440.0


In [4]:
from sklearn.model_selection import train_test_split

# Separar datos en train+val y test: 80% train+val, 20% test
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# De train+val, separamos en entrenamiento (train) y validación (val): 75% train, 25% val.
# Esto resulta en aproximadamente: 60% training, 20% validación y 20% test.
train_data, val_data = train_test_split(train_val_data, test_size=0.25, random_state=42)

print(f"Tamaño de train: {train_data.shape}")
print(f"Tamaño de validación: {val_data.shape}")
print(f"Tamaño de test: {test_data.shape}")

Tamaño de train: (60000, 20)
Tamaño de validación: (20000, 20)
Tamaño de test: (20000, 20)


In [ ]:
# test_data.to_parquet('data/test_data.parquet')

In [21]:
# Ajustar el modelo utilizando el conjunto de entrenamiento y validación (tuning_data).
# Aquí se asume que la columna objetivo se llama 'target'; cámbiala según tu dataset.
predictor = TabularPredictor(
    label='condition', 
    eval_metric='f1',
    path='runs/').fit(
    train_data=train_data, 
    tuning_data=val_data,
    # auto_stack=True,
    # presets='best_quality',
    # time_limit=60 * 2,
    excluded_model_types=['KNN', 'XT'],
    # use_bag_holdout=True,
    calibrate_decision_threshold=True,
    num_gpus=1 
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Mon Apr  7 19:01:38 UTC 2025
CPU Count:          32
Memory Avail:       47.49 GB / 60.42 GB (78.6%)
Disk Space Avail:   652.66 GB / 760.20 GB (85.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with fast inference sp

[1000]	valid_set's binary_logloss: 0.345788	valid_set's f1: 0.832507
[2000]	valid_set's binary_logloss: 0.342472	valid_set's f1: 0.837826
[3000]	valid_set's binary_logloss: 0.340193	valid_set's f1: 0.841805
[4000]	valid_set's binary_logloss: 0.34072	valid_set's f1: 0.841539


	0.8427	 = Validation score   (f1)
	13.34s	 = Training   runtime
	0.25s	 = Validation runtime
Fitting model: LightGBM ...
	Training LightGBM with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] bin size 799 cannot run on GPU
	0.8342	 = Validation score   (f1)
	1.65s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.8183	 = Validation score   (f1)
	0.62s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8203	 = Validation score   (f1)
	0.65s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost ...
	Training CatBoost with GPU, note that this may negatively impact model quality compared to CPU training.
	0.8386	 = Validation score   (f1)
	17.36s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 0: early stopping
	0.667	 = Validation score   (f1)
	16.87s	 = Training   runtime

In [9]:
predictor.evaluate(test_data, silent=True)

{'f1': 0.8436606291706387,
 'accuracy': 0.8524,
 'balanced_accuracy': 0.8530421032490454,
 'mcc': 0.7045332334198284,
 'roc_auc': 0.9294877173608864,
 'precision': 0.8265020234512815,
 'recall': 0.8615467820443483}

In [10]:
# Las métricas que se ven reflejadas es con respecto al test set (test_data)
extra_metric_list = ['log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy', 'mcc', 'f1', 'precision', 'recall']
predictor.leaderboard(test_data, extra_metrics=extra_metric_list).drop(['score_test', 'score_val', 'eval_metric'], axis=1)

/home/lynn/.cache/pypoetry/virtualenvs/venv-docker-fastapi-T9sC2B74-py3.12/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


,model,log_loss,roc_auc,accuracy,balanced_accuracy,mcc,f1,precision,recall,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.337520,0.929488,0.85240,0.853042,0.704533,0.843661,0.826502,0.861547,0.549652,0.386030,76.063146,0.001598,0.002679,0.552549,2,True,10
1,LightGBMXT,-0.345598,0.927027,0.84835,0.848631,0.695983,0.838610,0.825304,0.852353,0.282144,0.237617,13.785476,0.282144,0.237617,13.785476,1,True,1
2,LightGBMLarge,-0.338118,0.928878,0.84860,0.848484,0.696059,0.837971,0.829186,0.846944,0.027948,0.024234,2.757916,0.027948,0.024234,2.757916,1,True,9
3,CatBoost,-0.335216,0.929766,0.84835,0.848077,0.695411,0.837346,0.830355,0.844456,0.067096,0.035554,21.999825,0.067096,0.035554,21.999825,1,True,5
4,LightGBM,-0.344907,0.925648,0.84285,0.842651,0.684460,0.831700,0.823542,0.840022,0.021517,0.012156,1.674771,0.021517,0.012156,1.674771,1,True,2
5,NeuralNetTorch,-0.380405,0.912659,0.83125,0.833476,0.665072,0.825410,0.790997,0.862953,0.040650,0.039433,36.308446,0.040650,0.039433,36.308446,1,True,8
6,XGBoost,-0.357478,0.919277,0.83405,0.834469,0.667547,0.823935,0.808453,0.840022,0.050694,0.033938,4.562641,0.050694,0.033938,4.562641,1,True,7
7,RandomForestGini,-0.461104,0.908352,0.82990,0.829472,0.658292,0.817431,0.811162,0.823797,0.182109,0.047421,0.626065,0.182109,0.047421,0.626065,1,True,3
8,RandomForestEntr,-0.452861,0.909023,0.82925,0.828784,0.656954,0.816644,0.810768,0.822607,0.130215,0.046514,0.658934,0.130215,0.046514,0.658934,1,True,4
9,NeuralNetFastAI,-0.514539,0.868638,0.75360,0.737031,0.537080,0.660091,0.910908,0.517577,0.105776,0.094864,16.814670,0.105776,0.094864,16.814670,1,True,6


In [11]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 19 features using 5000 rows with 5 shuffle sets...
	34.25s	= Expected runtime (6.85s per shuffle set)
	11.31s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
listing_type,0.198833,0.008279,3.599431e-07,5,0.215880,0.181785
city,0.082342,0.007175,6.849196e-06,5,0.097116,0.067568
stock_quantity,0.050745,0.005597,1.747104e-05,5,0.062269,0.039222
available_quantity,0.040255,0.004620,2.045457e-05,5,0.049767,0.030743
total_amount,0.033857,0.002851,5.980549e-06,5,0.039728,0.027986
date_difference_hr,0.026107,0.004707,1.214776e-04,5,0.035798,0.016415
shipping_mode,0.025236,0.002483,1.110723e-05,5,0.030350,0.020123
time_difference_hr,0.024960,0.001687,2.486760e-06,5,0.028432,0.021487
state,0.012104,0.000924,4.047894e-06,5,0.014007,0.010201
attribute_id,0.009125,0.001959,2.400065e-04,5,0.013159,0.005092


In [6]:
import numpy as np
np.random.seed(77)

posiciones = np.random.randint(0, test_data.shape[0], size=7)
test_data.iloc[posiciones]

,condition,state,city,local_pickup,free_shipping,shipping_mode,listing_type,buying_mode,attribute_group_id,attribute_group,attribute_id,status,accepts_mercadopago,currency,automatic_relist,stock_quantity,available_quantity,total_amount,date_difference_hr,time_difference_hr
76294,used,capital federal,caba,True,False,me2,bronze,buy_it_now,None,None,None,paused,True,ars,False,2,2,0.0,483.279444,1440.0
72859,new,capital federal,flores,False,False,me2,bronze,buy_it_now,None,None,None,active,True,ars,False,2,2,0.0,1222.960278,1440.0
97118,new,capital federal,capital,True,False,me2,bronze,buy_it_now,None,None,None,active,True,ars,False,4,4,0.0,354.253889,1440.0
65706,new,capital federal,palermo,True,False,me2,bronze,buy_it_now,None,None,None,active,True,ars,False,3,3,0.0,0.001111,1440.0
14737,new,capital federal,bs as,True,False,not_specified,bronze,buy_it_now,None,None,None,active,True,ars,False,5,4,275.0,618.193333,1440.0
18670,new,capital federal,tribunales,True,False,me2,bronze,buy_it_now,None,None,None,active,True,ars,False,6,6,0.0,499.996389,1440.0
24824,used,capital federal,capital,True,False,not_specified,bronze,buy_it_now,None,None,None,active,True,ars,False,3,3,0.0,0.001111,1440.0


In [36]:
test_data.iloc[[0, 3], 1:].to_dict(orient='records')

[{'state': 'santa fe',
  'city': 'rosario',
  'local_pickup': False,
  'free_shipping': False,
  'shipping_mode': 'not_specified',
  'listing_type': 'bronze',
  'buying_mode': 'buy_it_now',
  'attribute_group_id': None,
  'attribute_group': None,
  'attribute_id': None,
  'status': 'active',
  'accepts_mercadopago': True,
  'currency': 'ars',
  'automatic_relist': False,
  'stock_quantity': 600,
  'available_quantity': 600,
  'total_amount': 0.0,
  'date_difference_hr': 72.39888888888889,
  'time_difference_hr': 1440.0},
 {'state': 'capital federal',
  'city': 'capital federal',
  'local_pickup': True,
  'free_shipping': False,
  'shipping_mode': 'me2',
  'listing_type': 'bronze',
  'buying_mode': 'buy_it_now',
  'attribute_group_id': None,
  'attribute_group': None,
  'attribute_id': None,
  'status': 'active',
  'accepts_mercadopago': True,
  'currency': 'ars',
  'automatic_relist': False,
  'stock_quantity': 1,
  'available_quantity': 1,
  'total_amount': 0.0,
  'date_difference_hr'

In [24]:
predictor = TabularPredictor.load('runs/')
predictor.predict(datapoint)

76294     new
72859     new
97118     new
65706     new
14737     new
18670     new
24824    used
Name: condition, dtype: object

In [7]:
datapoint = test_data.iloc[posiciones, 1:]
predictor.predict(datapoint)

NameError: name 'predictor' is not defined

In [ ]:
loaded_predictor = TabularPredictor.load('runs/')
loaded_predictor.predict(datapoint)

In [22]:
loaded_predictor = TabularPredictor.load('runs/')
loaded_predictor.predict(datapoint)

76294     new
72859     new
97118     new
65706     new
14737     new
18670     new
24824    used
Name: condition, dtype: object

In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.calibration import CalibratedClassifierCV

# Suponemos que 'predictor' es el modelo ya entrenado de AutoGluon
# y 'val_data' es el DataFrame de validación con la columna 'condition'.
X_val = val_data.drop(columns=['condition'])
y_val = val_data['condition']

class AutoGluonWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, predictor):
        self.predictor = predictor
        self.classes_ = None

    def fit(self, X, y):
        # Establecemos las clases para que el objeto se considere entrenado
        self.classes_ = np.unique(y)
        return self

    def predict_proba(self, X):
        # Se obtiene la salida del predictor y nos aseguramos de que sea un numpy array
        proba = self.predictor.predict_proba(X)
        # Si es un DataFrame, convertimos a numpy array:
        if isinstance(proba, pd.DataFrame):
            return proba.values
        else:
            return np.array(proba)

    def predict(self, X):
        proba = self.predictor.predict_proba(X)
        # Si es un DataFrame, se busca el índice de la columna con mayor probabilidad
        if isinstance(proba, pd.DataFrame):
            return proba.idxmax(axis=1).values
        else:
            return np.argmax(proba, axis=1)

# Creamos y "ajustamos" el wrapper para registrar las clases
wrapped_predictor = AutoGluonWrapper(predictor)
wrapped_predictor.fit(X_val, y_val)  # Esto asigna self.classes_

# Ahora usamos CalibratedClassifierCV para recalibrar las probabilidades
calibrated_clf = CalibratedClassifierCV(wrapped_predictor, method='isotonic', cv='prefit', n_jobs=-1)

# Calibramos utilizando el conjunto de validación
calibrated_clf.fit(X_val, y_val)

In [ ]:
pd.DataFrame(calibrated_clf.predict_proba(X_test), columns=calibrated_clf.classes_)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import brier_score_loss

# Supongamos que tienes los siguientes conjuntos:
X_test = test_data.drop(columns=['condition'])
y_test = test_data['condition']

# Obtener las probabilidades del modelo sin calibrar.
# En tal caso, se extrae la probabilidad de la clase positiva (columna 1).
uncalibrated_probs = predictor.predict_proba(X_test).iloc[:, 1].values

# Obtener las probabilidades del modelo calibrado.
# En este caso, el método predict_proba del calibrador devuelve un array.
calibrated_probs = calibrated_clf.predict_proba(X_test)[:, 1]

# Calcular el puntaje Brier: menor Brier score indica mejor calibración.
brier_uncal = brier_score_loss(y_test, uncalibrated_probs, pos_label=1)
brier_cal = brier_score_loss(y_test, calibrated_probs, pos_label=1)
print("Brier score (sin calibrar):", brier_uncal)
print("Brier score (calibrado):", brier_cal)

In [ ]:
p = predictor.load("./AutogluonModels/ag-20250413_193053")
p.evaluate(test_data, silent=True)